In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Time
import time

# Random and os for reproducibility
import random
import os

# Processing results
import json

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
import tensorflow as tf

# Processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

# Making keras compatible with scikit learn api
# https://scikit-learn.org/stable/developers/develop.html
from sklearn.base import RegressorMixin

2024-07-13 19:01:59.405622: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-13 19:01:59.409087: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-13 19:01:59.483974: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-13 19:01:59.485177: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-07-13 19:02:00.813533: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<IPython.core.display.Javascript object>

# Functions and definitions

In [3]:
def plot_predictions(model, df, scaler, index, date_column, x, y):
    dates = df[date_column].values
    test_series = pd.DataFrame({"ccs28": y}, index=pd.to_datetime(dates))
    pred_series = model.predict(scaler.transform(x)).squeeze()
    pred_series = pd.DataFrame({"ccs28-pred": pred_series}, index=pd.to_datetime(dates))

    fig, ax = plt.subplots(1, 1, sharex=True, sharey=True, figsize=(15, 7))

    test_series.plot(ax=ax)
    ax.axvline(test_series.index[index], color="r")  # end of train dataset
    pred_series[index:].plot(ax=ax)
    ax.grid(which="both")
    ax.legend(
        ["train and test series", "end of train series", "predicted"], loc="upper left"
    )
    ax.set_xlabel("Dates", labelpad=20, fontsize=15)
    ax.set_ylabel("Compressive Strength - MPa", labelpad=20, fontsize=15)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    plt.show()

<IPython.core.display.Javascript object>

In [4]:
def plot_scores_box_plot(scores, repeats, n_splits):
    plt.figure(figsize=(15, 8))
    plt.boxplot(
        scores.reshape((repeats, n_splits)),
        labels=[str(r) for r in range(1, repeats + 1)],
        showmeans=True,
    )
    plt.ylabel("RMSE", labelpad=20, fontsize=15)
    plt.xlabel("Repeats", labelpad=20, fontsize=15)
    plt.show()

<IPython.core.display.Javascript object>

In [5]:
class BlockingTimeSeriesSplit:
    def __init__(self, n_splits, train_size):
        self.n_splits = n_splits
        self.train_size = train_size

    def get_n_splits(self, X, y, groups):
        return self.n_splits

    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(self.train_size * (stop - start)) + start
            yield indices[start:mid], indices[mid + margin : stop]


# Reference: https://goldinlocks.github.io/Time-Series-Cross-Validation/

<IPython.core.display.Javascript object>

## Helper functions for blocked time series cross validation

### Train test split

In [6]:
def split_by_periods(x, y, train_period, test_period):
    datasets = []
    i = 0
    max_samples = x.shape[0]

    for _ in range(0, max_samples // (train_period + test_period)):
        # Splitting the data into train/test sets
        x_train = x[i : i + train_period].copy()
        y_train = y[i : i + train_period].copy()
        x_test = x[i + train_period : i + train_period + test_period].copy()
        y_test = y[i + train_period : i + train_period + test_period].copy()

        datasets.append(
            {
                "x_train": x_train,
                "y_train": y_train,
                "x_test": x_test,
                "y_test": y_test,
            }
        )
        # Increments the index for the next period of time
        i += train_period + test_period

    return datasets

<IPython.core.display.Javascript object>

In [7]:
def split_by_dates(x, y, train_period, test_period, dates):
    datasets = []
    dates = dates[: x.shape[0]].copy()
    dates_unique = dates.copy().unique()
    i = 0
    max_samples = dates_unique.shape[0]

    for _ in range(0, max_samples // (train_period + test_period)):
        # Splitting the Train Set
        start_date_train = dates_unique[i]
        end_date_train = dates_unique[i + train_period]
        idx_train = dates[(dates >= start_date_train) & (dates < end_date_train)].index
        x_train = x.loc[idx_train].copy()
        y_train = y[idx_train].copy()

        # Splitting the Test Set
        start_date_test = dates_unique[i + train_period]
        end_date_test = dates_unique[i + train_period + test_period]
        idx_test = dates[(dates >= start_date_test) & (dates < end_date_test)].index
        x_test = x.loc[idx_test].copy()
        y_test = y[idx_test].copy()

        datasets.append(
            {
                "x_train": x_train,
                "y_train": y_train,
                "x_test": x_test,
                "y_test": y_test,
            }
        )

        i += train_period + test_period

    return datasets

<IPython.core.display.Javascript object>

In [8]:
def train_test_split_blocked_ts(x, y, train_period, test_period, dates=None):
    """
    Split the input data into train-test datasets based on train and test periods.

    Args:
        x (pd.DataFrame): Input features.
        y (np.Array): Target values.
        train_period (int): Length of the training period.
        test_period (int): Length of the testing period.
        dates (pd.Series): Optional date information.

    Returns:
        List[dict]: A list of dictionaries, each containing 'x_train', 'y_train', 'x_test', and 'y_test'.
    """
    if dates is None:
        return split_by_periods(x, y, train_period, test_period)
    else:
        return split_by_dates(x, y, train_period, test_period, dates)

<IPython.core.display.Javascript object>

### Data preprocessing

In [9]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [10]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [11]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [12]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(**estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

### Blocking time series cross validation

In [13]:
def repeated_blocking_time_series(
    Estimator,
    Transform,
    Imputer,
    x,
    y,
    train_period,
    test_period,
    dates=None,
    repeats=10,
    estimator_params=None,
    imputer_params=None,
):
    """
    Perform repeated cross-validation with blocked time series data.

    Args:
        Estimator: Machine learning model class.
        Transform: Data transformation method.
        Imputer: Data imputation method.
        x: Input features.
        y: Target values.
        train_period: Length of the training period.
        test_period: Length of the testing period.
        dates: Optional date information.
        repeats: Number of repetitions.
        estimator_params: Parameters for the model.
        imputer_params: Parameters for data imputation.

    Returns:
        list: List of dictionaries containing evaluation metrics for each repetition.
    """

    results = []
    max_samples = x.shape[0]

    # Splitting the data into train/test sets
    datasets = train_test_split_blocked_ts(x, y, train_period, test_period, dates)

    for _ in range(repeats):
        scores = []

        for dataset in datasets:
            dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)
            score = train_and_evaluate_model(Estimator, dataset, estimator_params)
            scores.append(score)

        # After every iteration metrics results are appended together
        scores_final = {key: [] for key, _ in scores[0].items()}
        for scores_dict in scores:
            for key, value in scores_dict.items():
                scores_final[key] += [value]
        results.append(scores_final)
    return results

<IPython.core.display.Javascript object>

In [14]:
def print_scores(scores, METRICS, METRICS_DICT):
    for phase in ["train", "test"]:
        print("******")
        print(f"[{phase.upper()}]")
        print("******")
        for metric in METRICS:
            name = METRICS_DICT[metric]
            print(
                f"{name}: %.3f (%.3f)"
                % (
                    np.mean(scores[f"{phase}_" + metric]),
                    np.std(scores[f"{phase}_" + metric]),
                )
            )
        print("\n======================\n")

<IPython.core.display.Javascript object>

In [15]:
def score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred):
    """
    Calculate and return regression metrics for both training and testing sets.

    Args:
        y_train (array-like): True target values for the training set.
        y_train_pred (array-like): Predicted target values for the training set.
        y_test (array-like): True target values for the testing set.
        y_test_pred (array-like): Predicted target values for the testing set.

    Returns:
        dict: Dictionary containing regression metrics.
    """
    TRAIN_RMSE = mean_squared_error(y_true=y_train, y_pred=y_train_pred, squared=False)
    TRAIN_MAE = mean_absolute_error(y_true=y_train, y_pred=y_train_pred)
    TRAIN_MAPE = mean_absolute_percentage_error(y_true=y_train, y_pred=y_train_pred)
    TRAIN_R2 = r2_score(y_true=y_train, y_pred=y_train_pred)

    TEST_RMSE = mean_squared_error(y_true=y_test, y_pred=y_test_pred, squared=False)
    TEST_MAE = mean_absolute_error(y_true=y_test, y_pred=y_test_pred)
    TEST_MAPE = mean_absolute_percentage_error(y_true=y_test, y_pred=y_test_pred)
    TEST_R2 = r2_score(y_true=y_test, y_pred=y_test_pred)

    scores = {
        "train_neg_root_mean_squared_error": TRAIN_RMSE,
        "train_neg_mean_absolute_error": TRAIN_MAE,
        "train_neg_mean_absolute_percentage_error": TRAIN_MAPE,
        "train_r2": TRAIN_R2,
        "test_neg_root_mean_squared_error": TEST_RMSE,
        "test_neg_mean_absolute_error": TEST_MAE,
        "test_neg_mean_absolute_percentage_error": TEST_MAPE,
        "test_r2": TEST_R2,
    }
    return scores

<IPython.core.display.Javascript object>

In [16]:
def fill_results_dict(results_dict, scores):
    num_samples = len(scores["test_r2"])

    # Create a DataFrame by copying the results_dict for each sample
    df_results = pd.DataFrame([results_dict.copy()] * num_samples)

    # Define a mapping of metric names between the input scores and the DataFrame
    metric_mapping = {
        "train_neg_root_mean_squared_error": "RMSE Train",
        "test_neg_root_mean_squared_error": "RMSE Test",
        "train_neg_mean_absolute_error": "MAE Train",
        "test_neg_mean_absolute_error": "MAE Test",
        "train_neg_mean_absolute_percentage_error": "MAPE Train",
        "test_neg_mean_absolute_percentage_error": "MAPE Test",
        "train_r2": "R2 Train",
        "test_r2": "R2 Test",
    }

    # Iterate through the mapping and fill the DataFrame
    for score_key, df_column_name in metric_mapping.items():
        if score_key in scores:
            if "R2" not in df_column_name:
                df_results[df_column_name] = np.abs(scores[score_key])
            else:
                df_results[df_column_name] = scores[score_key]

    return df_results

<IPython.core.display.Javascript object>

## Model Definition

In [17]:
class MLP(RegressorMixin):
    def __init__(self):
        self.model = self.get_model()
        self.batch_size = 16
        self.epochs = 300
        self.verbose = 0

    def fit(self, X=None, y=None):
        self.model.fit(
            X, y, batch_size=self.batch_size, epochs=self.epochs, verbose=self.verbose
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(units=16, activation="relu")),
        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            # optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

# Settings for Reproducibility

In [18]:
def set_seeds():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    os.environ["PYTHONHASHSEED"] = str(SEED)
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)


# https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed

<IPython.core.display.Javascript object>

In [19]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [20]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_i",
    "Features": "Chemical + Mineralogical + Physical",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "MLP",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [21]:
df = pd.read_csv("../../../../../../data/processed/partner_i-Oficial/cement-shipping.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we use all available features

In [22]:
df_copy = df.drop(
    [  # Removing One-Hot encoding variables
        "Cement_Type",
        
    ],
    axis=1,
).copy()

<IPython.core.display.Javascript object>

<h2>1. Dataset: df_copy</h2> <br>In this dataset all features are used.

In [23]:
y = df_copy.pop("CS28").values
x = df_copy.drop(["Date"], axis=1)
dates = df["Date"].copy()

<IPython.core.display.Javascript object>

# 1. Multi Layer Perceptron

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 37)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>5<br>
    1. 5 folds of 246 samples each
    2. 80% train (988 samples each fold)
    3. 20% test (246 samples each fold)
<b>Total:</b> 100 models<br>

In [24]:
set_seeds()
start = time.time()

repeats = 3
n_splits = 5
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", MLP()),
    ]
)
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=1,
    return_train_score=True,
)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Repeated KFold"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

2024-07-13 19:02:06.781540: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-13 19:02:06.781586: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: inspirada
2024-07-13 19:02:06.781594: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: inspirada
2024-07-13 19:02:06.781816: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.183.1
2024-07-13 19:02:06.781849: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.183.1
2024-07-13 19:02:06.781854: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.183.1


Repeated Cross Validation:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.097 (0.057)
MAE: -0.866 (0.047)
MAPE: -0.020 (0.001)
R2: 0.953 (0.005)


******
[TEST]
******
RMSE: -1.187 (0.096)
MAE: -0.924 (0.075)
MAPE: -0.021 (0.002)
R2: 0.944 (0.008)


Minutes Elapsed:  8.031782738367717


<IPython.core.display.Javascript object>

In [25]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features Model Cross Validation Timesteps  \
                                                                          
0  Chemical + Mineralogical + Physical   MLP   Repeated KFold       NaN   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  1.187407  0.095578  0.924239  0.075126  0.021415  0.001905  0.944114   

             
        std  
0  0.007868

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [26]:
set_seeds()
start = time.time()

repeats = 3
n_splits = 5
train_size = 0.8
scores_final = None

for _ in range(repeats):
    pipeline = Pipeline(
        [
            ("imputer", SimpleImputer(strategy="median")),
            ("transformer", StandardScaler()),
            ("estimator", MLP()),
        ]
    )
    cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)
    scores = cross_validate(
        pipeline,
        x,
        y,
        scoring=METRICS,
        cv=cv,
        # n_jobs=None,
        return_train_score=True,
    )
    if scores_final is None:
        scores_final = {key: [] for key, _ in scores.items()}

    for key, value in scores.items():
        scores_final[key] += [value]


print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores_final, METRICS, METRICS_DICT)

scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
results_dict_copy[
    "Cross Validation Params"
] = '{"N_Splits": 5, "Repeats": 3, "train_size": 0.8}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -3.552 (0.427)
MAE: -2.804 (0.330)
MAPE: -0.065 (0.007)
R2: 0.497 (0.127)


******
[TEST]
******
RMSE: -6.289 (0.845)
MAE: -4.969 (0.769)
MAPE: -0.116 (0.018)
R2: -0.644 (0.340)


Minutes Elapsed:  2.0564977407455443


<IPython.core.display.Javascript object>

In [27]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features Model            Cross Validation  \
                                                                           
0  Chemical + Mineralogical + Physical   MLP  Blocking Time Series Split   
1  Chemical + Mineralogical + Physical   MLP              Repeated KFold   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  6.288996  0.845163  4.969017  0.769403  0.116256  0.018108 -0.644155   
1  1.187407  0.095578  0.924239  0.075126  0.021415  0.001905  0.944114   

             
        std  
0  0.339835  
1  0.007868

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [28]:
set_seeds()
start = time.time()

scores_final = None
repeats = 3
n_splits = 5
gap = 0

for _ in range(repeats):
    pipeline = Pipeline(
        [
            ("imputer", SimpleImputer(strategy="median")),
            ("transformer", StandardScaler()),
            ("estimator", MLP()),
        ]
    )
    cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits)

    scores = cross_validate(
        pipeline,
        x,
        y,
        scoring=METRICS,
        cv=cv,
        # n_jobs=-1,
        return_train_score=True,
    )
    if scores_final is None:
        scores_final = {key: [] for key, _ in scores.items()}
    for key, value in scores.items():
        scores_final[key] += [value]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores_final, METRICS, METRICS_DICT)

# Saving the results
scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Time Series Split"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3, "Gap": 0}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.932 (0.958)
MAE: -1.524 (0.762)
MAPE: -0.036 (0.018)
R2: 0.805 (0.201)


******
[TEST]
******
RMSE: -3.829 (2.946)
MAE: -2.924 (2.143)
MAPE: -0.069 (0.051)
R2: 0.058 (1.375)


Minutes Elapsed:  5.2519904255867


<IPython.core.display.Javascript object>

In [29]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features Model            Cross Validation  \
                                                                           
0  Chemical + Mineralogical + Physical   MLP  Blocking Time Series Split   
1  Chemical + Mineralogical + Physical   MLP              Repeated KFold   
2  Chemical + Mineralogical + Physical   MLP           Time Series Split   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  6.288996  0.845163  4.969017  0.769403  0.116256  0.018108 -0.644155   
1  1.187407  0.095578  0.924239  0.075126  0.021415  0.001905  0.944114   
2  3.828779  2.946002  2.923995  2.142933  0.068667  0.050860  0.057774   

             
        std  
0  0.339835  
1  0.007868  
2  1.375247

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [30]:
set_seeds()
start = time.time()

test_size = 0.2
repeats = 3
scores_final = None

print("Out of time Cross Val:")
print(f"Repeats: {repeats}")
print(f"Train: {80}%", f"Test: {20}%")
print()


for _ in range(repeats):
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=test_size, random_state=SEED, shuffle=False
    )

    pipeline = Pipeline(
        [
            ("imputer", SimpleImputer(strategy="median")),
            ("transformer", StandardScaler()),
            ("estimator", MLP()),
        ]
    )

    pipeline.fit(x_train, y_train)

    y_train_pred = pipeline.predict(x_train)
    y_test_pred = pipeline.predict(x_test)

    scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

    if scores_final is None:
        scores_final = {key: [] for key, _ in scores.items()}

    for key, value in scores.items():
        scores_final[key] += [value]

# Saving the results
results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Out of time Split"
results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(
    results_dict_copy, {key: value for key, value in scores_final.items()}
)
results_to_save.append(df_results)

print_scores(scores_final, METRICS, METRICS_DICT)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Out of time Cross Val:
Repeats: 3
Train: 80% Test: 20%



******
[TRAIN]
******
RMSE: 1.045 (0.028)
MAE: 0.815 (0.021)
MAPE: 0.019 (0.000)
R2: 0.957 (0.002)


******
[TEST]
******
RMSE: 1.277 (0.010)
MAE: 1.020 (0.007)
MAPE: 0.024 (0.000)
R2: 0.934 (0.001)


Minutes Elapsed:  1.543573053677877


<IPython.core.display.Javascript object>

In [31]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features Model            Cross Validation  \
                                                                           
0  Chemical + Mineralogical + Physical   MLP  Blocking Time Series Split   
1  Chemical + Mineralogical + Physical   MLP           Out of time Split   
2  Chemical + Mineralogical + Physical   MLP              Repeated KFold   
3  Chemical + Mineralogical + Physical   MLP           Time Series Split   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  6.288996  0.845163  4.969017  0.769403  0.116256  0.018108 -0.644155   
1  1.276626  0.009603  1.019811  0.007371  0.023555  0.000208  0.934155   
2  1.187407  0.095578  0.924239  0.075126  0.021415  0.001905  0.944114   
3  3.828779  2.946002  2.923995  2.142933  0.068667  0.050860  0.057774   

             
        std  
0  0.339835  
1  0.000991  
2  0.007868  
3  1.375247

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [32]:
path = (
    "../../../../../../reports/results/local_models/partner_i-oficial/all_cements/full/"
)
filename = "mlp_results_full_7.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [33]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../../reports/results/local_models/partner_i-oficial/all_cements/grouped/"
filename = "mlp_results_grouped_7.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,  # header=df_results_to_save.columns
)

<IPython.core.display.Javascript object>